In [1]:
#This is for top insurence state wise data
import pandas as pd
import json
import os

#This is to direct the path to get the data as states

path=r"D:/BONEYS/WEB/PYTHON/Project/PhonePe/data/top/insurance/country/india/state"
Top_state_list=os.listdir(path)

#This is to extract the data's to create a dataframe
clm={'State':[], 'Year':[],'Quater':[],'Entity_name':[], 'Insurance_count':[], 'Insurance_amount':[]}

for i in Top_state_list:
    p_i=path+"/"+i
    Agg_yr=os.listdir(p_i)
    #delhi
    for j in Agg_yr:
        p_j=p_i+"/"+j
        #2018
        Agg_yr_list=os.listdir(p_j)
        for k in Agg_yr_list:
            p_k=p_j+"/"+k 
            #1.json
            Data=open(p_k,'r')
            D=json.load(Data)
            for z in D['data']['districts']:
              Name=z['entityName']
              count=z['metric']['count']
              amount=z['metric']['amount']
              clm['Entity_name'].append(Name)
              clm['Insurance_count'].append(count)
              clm['Insurance_amount'].append(amount)
              clm['State'].append(i)
              clm['Year'].append(j)
              clm['Quater'].append(int(k.strip('.json')))
#Succesfully created a dataframe
Top_Insu_state=pd.DataFrame(clm)
print(Top_Insu_state)

                          State  Year  Quater               Entity_name  \
0     andaman-&-nicobar-islands  2020       2                  nicobars   
1     andaman-&-nicobar-islands  2020       2             south andaman   
2     andaman-&-nicobar-islands  2020       3             south andaman   
3     andaman-&-nicobar-islands  2020       3                  nicobars   
4     andaman-&-nicobar-islands  2020       3  north and middle andaman   
...                         ...   ...     ...                       ...   
5603                west-bengal  2024       4         paschim bardhaman   
5604                west-bengal  2024       4         paschim medinipur   
5605                west-bengal  2024       4                     malda   
5606                west-bengal  2024       4                     nadia   
5607                west-bengal  2024       4           purba bardhaman   

      Insurance_count  Insurance_amount  
0                   3             565.0  
1              

In [2]:
#You need to map your State names to the names used in the GeoJSON file (properties.ST_NM). Otherwise you will get blank figure.
state_name_map = {
    'andaman-&-nicobar-islands': 'Andaman & Nicobar Island',
    'andhra-pradesh': 'Andhra Pradesh',
    'arunachal-pradesh': 'Arunanchal Pradesh',
    'assam': 'Assam',
    'bihar': 'Bihar',
    'chandigarh': 'Chandigarh',
    'chhattisgarh': 'Chhattisgarh',
    'dadra-&-nagar-haveli-&-daman-&-diu': 'Dadra and Nagar Haveli',
    'delhi': 'NCT of Delhi',
    'goa': 'Goa',
    'gujarat': 'Gujarat',
    'haryana': 'Haryana',
    'himachal-pradesh': 'Himachal Pradesh',
    'jammu-&-kashmir': 'Jammu & Kashmir',
    'jharkhand': 'Jharkhand',
    'karnataka': 'Karnataka',
    'kerala': 'Kerala',
    'ladakh': 'Ladakh',
    'lakshadweep': 'Lakshadweep',
    'madhya-pradesh': 'Madhya Pradesh',
    'maharashtra': 'Maharashtra',
    'manipur': 'Manipur',
    'meghalaya': 'Meghalaya',
    'mizoram': 'Mizoram',
    'nagaland': 'Nagaland',
    'odisha': 'Odisha',
    'puducherry': 'Puducherry',
    'punjab': 'Punjab',
    'rajasthan': 'Rajasthan',
    'sikkim': 'Sikkim',
    'tamil-nadu': 'Tamil Nadu',
    'telangana': 'Telangana',
    'tripura': 'Tripura',
    'uttar-pradesh': 'Uttar Pradesh',
    'uttarakhand': 'Uttarakhand',
    'west-bengal': 'West Bengal'
}
Top_Insu_state['State'] = Top_Insu_state['State'].str.strip().str.lower()
Top_Insu_state['State'] = Top_Insu_state['State'].replace(state_name_map)

In [ ]:
#creating table to store the above data
import mysql.connector
connection = mysql.connector.connect(
    host="localhost",
    user="xxx", #give your user name. Mostly its root
    password="xxx", #give your password
    database="xxx" #give any database name you want
)
cursor=connection.cursor()
query1 = """
        create table Top_Insurance_State(
        State varchar(100),
        Year int,
        Quater int,
        Entity_name varchar(100),
        Insurance_count int,
        Insurance_amount float)
        """
cursor.execute(query1)

In [5]:
#storing the above in mysql database
#all data must be tuple and must be a list of tuples
data=[]
for index in Top_Insu_state.index:
    row = Top_Insu_state.loc[index]
    row = row.values
    row = row[0],int(row[1]),int(row[2]),row[3],int(row[4]),float(row[5])
    data.append(row)
query2 = "insert into Top_Insurance_State values (%s,%s,%s,%s,%s,%s)"
cursor.executemany(query2,data)
connection.commit()

In [3]:
#doing data cleaning for Top_Insu_state
from sklearn.preprocessing import LabelEncoder
#first step check and remove null values
Top_Insu_state.isna().sum()
Top_Insu_state.info()
#Second do encoding and dummies to make all values int
enc_col = ['Entity_name']
dummy_col = ['Year','State','Quater']
encoder = LabelEncoder()
Top_Insu_state['encoded_entity_name'] = encoder.fit_transform(Top_Insu_state['Entity_name'])
Top_Insu_state = pd.get_dummies(Top_Insu_state,columns=dummy_col,dtype=int)
#third removing outliners
Top_Insu_state['insurance_count_std']=(Top_Insu_state['Insurance_count']-Top_Insu_state['Insurance_count'].mean())/Top_Insu_state['Insurance_count'].std()
Top_Insu_state['insurance_amount_std']=(Top_Insu_state['Insurance_amount']-Top_Insu_state['Insurance_amount'].mean())/Top_Insu_state['Insurance_amount'].std()
condition1 = Top_Insu_state['insurance_count_std']<=3
condition2 = Top_Insu_state['insurance_count_std']>=-3
condition3 = Top_Insu_state['insurance_amount_std']<=3
condition4 = Top_Insu_state['insurance_amount_std']>=-3
Top_Insu_state = Top_Insu_state[(condition1) & (condition2) & (condition3) & (condition4)]
print(Top_Insu_state)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5608 entries, 0 to 5607
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   State             5608 non-null   object 
 1   Year              5608 non-null   object 
 2   Quater            5608 non-null   int64  
 3   Entity_name       5608 non-null   object 
 4   Insurance_count   5608 non-null   int64  
 5   Insurance_amount  5608 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 263.0+ KB
                   Entity_name  Insurance_count  Insurance_amount  \
0                     nicobars                3             565.0   
1                south andaman                3             795.0   
2                south andaman               35           13651.0   
3                     nicobars                5            1448.0   
4     north and middle andaman                1             281.0   
...                        ...          

In [7]:
#plotting aggregated_state_insurance in the form of india map using plotly
import plotly.express as px

fig = px.choropleth(
    Top_Insu_state,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    featureidkey='properties.ST_NM',
    locations='State',
    color='Insurance_count',
    color_continuous_scale='Reds'
)

fig.update_geos(fitbounds="locations", visible=False)

fig.show()

In [4]:
#This is for top insurence state wise data for 2020
import pandas as pd
import json
import os

#This is to direct the path to get the data as states

path=r"D:/BONEYS/WEB/PYTHON/Project/PhonePe/data/top/insurance/country/india/2020"
Top_insu_2020_list=os.listdir(path)

#This is to extract the data's to create a dataframe
clm={'Quater':[],'Place_name':[], 'Insurance_count':[], 'Insurance_amount':[]}

for i in Top_insu_2020_list:
    p_i=path+"/"+i
    Data=open(p_i,'r')
    D=json.load(Data)
    for z in D['data']['districts']:
        Name=z['entityName']
        count=z['metric']['count']
        amount=z['metric']['amount']
        clm['Place_name'].append(Name)
        clm['Insurance_count'].append(count)
        clm['Insurance_amount'].append(amount)
        clm['Quater'].append(int(i.strip('.json')))
#Succesfully created a dataframe
Top_Insu_2020=pd.DataFrame(clm)
print(Top_Insu_2020)

    Quater          Place_name  Insurance_count  Insurance_amount
0        2     bengaluru urban            20040         3301769.0
1        2                pune             8699         1482628.0
2        2               thane             7559         1221018.0
3        2     mumbai suburban             5780          884439.0
4        2           hyderabad             4783          805518.0
5        2          rangareddy             4495          778492.0
6        2  medchal malkajgiri             4150          711890.0
7        2       visakhapatnam             3504          614212.0
8        2          aurangabad             2898          509460.0
9        2           anantapur             2739          485777.0
10       3     bengaluru urban            32192         6940765.0
11       3                pune            18176         4567561.0
12       3       visakhapatnam             7906         1892376.0
13       3               thane             6951         1718946.0
14       3